In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [3]:
# read data and skip 'date' colume
df = pd.read_csv('./dataset/netflix-prize-data/combined_data_1.txt', 
                 header = None, names = ['user', 'rating','date'], 
                 usecols = [0,1,2])

KeyboardInterrupt: 

In [3]:
def slice_movie(df):
    # find empty rows for slice movies 
    # dataset using movie id as user name for splite rating of each movies
    movies_interval = df[df['rating'].isna()]['user'].reset_index()
    movies_interval['index'] = movies_interval['index'].astype(int)
    
    # create list accroding to movie interval by np
    movie_id = 1
    movie_np = []
    last = None
    for i,r in movies_interval.iterrows():
        if last != None: 
            temp = np.full((1,r['index']-last-1), movie_id)
            movie_np = np.append(movie_np, temp)
            movie_id +=1
        last = r['index']
    # add last movie
    temp = np.full((1,len(df)-last-1), movie_id)
    movie_np = np.append(movie_np, temp)
    
    # removie movie slice row, and combine movie colume
    df = df[pd.notnull(df['rating'])]
    df['movie'] = movie_np.astype(int)
    
    return df

In [4]:
def get_users_df(df):
    # get user list and their rating count
    return df.groupby('user')['user'].agg(['count']).reset_index()

In [5]:
def filter_data_by_user(df,users_df, frac=0.1,count=50):
    # get test users
    test_users_df = users_df[users_df['count']>count].sample(frac=frac)
    # filter user by rating count
    cdf = df.merge(users_df, left_on='user',right_on='user')
    cdf = cdf[cdf['count']>count][['user','movie','rating']]
    
    train_set = cdf[~(cdf['user'].isin(test_users_df['user']))]
    test_set = cdf[cdf['user'].isin(test_users_df['user'])]
    return train_set, test_set

In [53]:
def filter_data_by_date(df,users_df, frac=0.1,count=100,year=2005,month=12,day=15):
    # get test users
    test_users_df = users_df[users_df['count']>count].sample(frac=frac)
    # filter user by rating count
    cdf = df.merge(users_df, on='user')
    cdf = cdf[cdf['count']>count][['user','movie','rating','date']]
    
#     train_set = cdf[~(cdf['user'].isin(test_users_df['user'])& (cdf['date']< pd.datetime(year,month,day)))]
    test_set = cdf[cdf['user'].isin(test_users_df['user'])& (cdf['date']> pd.datetime(year,month,day))]
    target_set = cdf[cdf['user'].isin(test_users_df['user'])]
    train_set = cdf[~(cdf.isin(test_set))]
    
    return train_set[pd.notnull(train_set['user'])], test_set, target_set

In [7]:
sliced_df = slice_movie(df)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [8]:

# print(sliced_df['date'][:5])
# print(sliced_df[sliced_df['date']> pd.datetime(2005,12,1)])

In [13]:
sliced_df['date']= pd.to_datetime(sliced_df['date'])
sliced_df['rating']=sliced_df['rating'].astype(float)
sliced_df['movie']=sliced_df['movie'].astype(int)
sliced_df['user']=sliced_df['user'].astype(int)
print(sliced_df)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


             user  rating       date  movie
1         1488844     3.0 2005-09-06      1
2          822109     5.0 2005-05-13      1
3          885013     4.0 2005-10-19      1
4           30878     4.0 2005-12-26      1
5          823519     3.0 2004-05-03      1
6          893988     3.0 2005-11-17      1
7          124105     4.0 2004-08-05      1
8         1248029     3.0 2004-04-22      1
9         1842128     4.0 2004-05-09      1
10        2238063     3.0 2005-05-11      1
11        1503895     4.0 2005-05-19      1
12        2207774     5.0 2005-06-06      1
13        2590061     3.0 2004-08-12      1
14           2442     3.0 2004-04-14      1
15         543865     4.0 2004-05-28      1
16        1209119     4.0 2004-03-23      1
17         804919     4.0 2004-06-10      1
18        1086807     3.0 2004-12-28      1
19        1711859     4.0 2005-05-08      1
20         372233     5.0 2005-11-23      1
21        1080361     3.0 2005-03-28      1
22        1245640     3.0 2005-1

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [54]:
users_df = get_users_df(sliced_df)
# train_set, test_set = filter_data_by_user(sliced_df,users_df,frac=0.1,count=50)
train_set, test_set, target_set = filter_data_by_date(sliced_df,users_df)

In [55]:
train_set

,user,movie,rating,date
0,1488844.0,1.0,3.0,2005-09-06
1,1488844.0,8.0,4.0,2005-05-12
2,1488844.0,17.0,2.0,2005-06-29
3,1488844.0,30.0,3.0,2005-04-13
4,1488844.0,44.0,3.0,2005-04-20
5,1488844.0,58.0,5.0,2005-04-20
6,1488844.0,76.0,3.0,2005-04-26
7,1488844.0,80.0,3.0,2005-06-02
8,1488844.0,81.0,3.0,2005-05-09
9,1488844.0,83.0,3.0,2005-07-19


In [57]:
train_set.to_csv(r'./dataset/cleaned_train_5.csv',index=False,sep=',')
test_set.to_csv(r'./dataset/cleaned_test_5.csv',index=False,sep=',')
target_set.to_csv(r'./dataset/cleaned_target_5.csv',index=False,sep=',')

In [79]:
MOVIE_NUM = len(train_set[['movie']].drop_duplicates() )
print(MOVIE_NUM)

4499

In [85]:
def make_sprase_matrix(df):

    users = df[['user']].drop_duplicates() #用户列表
    users = users['user'].astype(int)
    np_matrix = np.zeros((len(users), MOVIE_NUM))
    i = 0
    for user in users:
        user_list = df[df['user']== user] #获取全部用户评分
        np_matrix[i][user_list['movie'].astype(int) -1] = user_list['rating'] #按index插入评分
        i+=1
        if i%100==0:
            print("count: ",i)
    return np_matrix

In [84]:
sprase_train = make_sprase_matrix(train_set)
np.savetxt('dataset/sprase_train_5.txt',sprase_train)
sprase_test = make_sprase_matrix(test_set)
np.savetxt('dataset/sprase_test_5.txt',sprase_test)

In [86]:
sprase_target = make_sprase_matrix(target_set)
np.savetxt('dataset/sprase_target_5.txt',sprase_test)

count:  100
count:  200
count:  300
count:  400
count:  500
count:  600
count:  700
count:  800
count:  900
count:  1000
count:  1100
count:  1200
count:  1300
count:  1400
count:  1500
count:  1600
count:  1700
count:  1800
count:  1900
count:  2000
count:  2100
count:  2200
count:  2300
count:  2400
count:  2500
count:  2600
count:  2700
count:  2800
count:  2900
count:  3000
count:  3100
count:  3200
count:  3300
count:  3400
count:  3500
count:  3600
count:  3700
count:  3800
count:  3900
count:  4000
count:  4100
count:  4200
count:  4300
count:  4400
count:  4500
count:  4600
count:  4700
count:  4800
count:  4900
count:  5000
count:  5100
count:  5200
count:  5300
count:  5400
count:  5500
count:  5600
count:  5700
count:  5800
count:  5900
count:  6000
count:  6100
count:  6200
count:  6300
count:  6400
count:  6500
count:  6600
count:  6700
count:  6800
count:  6900
count:  7000
